# CCTV 낙상/누움 감지 데이터셋 딥러닝 학습 실습
이 노트북은 CCTV Incident Dataset (Fall and Lying Down Detection)을 활용하여 딥러닝 기반 객체 검출 및 분류 모델을 학습, 평가, 저장하는 전체 과정을 다룹니다.

---

## 1. 필요한 라이브러리 임포트
딥러닝 학습 및 데이터 처리를 위한 필수 라이브러리를 임포트합니다.

In [1]:
# 필수 라이브러리 임포트
import os
import matplotlib.pyplot as plt

## 2. 데이터셋 준비 및 불러오기
YOLO 학습을 위해 데이터셋의 구조를 확인하고, 필요한 경우 data.yaml 파일을 생성합니다.
- 데이터 경로: /home/clyde/dev_ws/deeplearning-repo-2/data/CCTV_Incident_Fall_Lying_Dataset/laying_dataset

In [2]:
# YOLO 학습을 위한 데이터셋 경로 지정 및 구조 확인
base_dir = '/home/clyde/dev_ws/deeplearning-repo-2/data/CCTV_Incident_Fall_Lying_Dataset/laying_dataset'
print('데이터셋 폴더 구조:')
for root, dirs, files in os.walk(base_dir):
    print(root, len(files), 'files')
    # 하위 폴더까지 모두 출력하려면 break 제거
    break

데이터셋 폴더 구조:
/home/clyde/dev_ws/deeplearning-repo-2/data/CCTV_Incident_Fall_Lying_Dataset/laying_dataset 3 files


## 2. 데이터셋 준비 및 train/val 자동 분리
- 원본 laying_dataset 폴더의 .png/.txt 파일을 기준으로, 정상 레이블만 골라 8:2로 분리합니다.
- images/train, images/val, labels/train, labels/val 폴더가 자동 생성됩니다.
- 레이블 형식 오류가 있는 파일은 복사하지 않습니다.

In [3]:
# [Step 1] 원본에서 train/val 자동 분리 (오류 없는 레이블만 사용)
# - 원본 laying_dataset 폴더의 .png/.txt 파일을 기준으로, 정상 레이블만 골라 8:2로 분리합니다.
# - images/train, images/val, labels/train, labels/val 폴더가 자동 생성됩니다.
# - 레이블 형식 오류가 있는 파일은 복사하지 않습니다.
import shutil, random

# 원본 이미지/레이블 경로
img_dir = os.path.join(base_dir, 'images')
lbl_dir = os.path.join(base_dir, 'labels')

# .png 파일 기준으로 분리
all_imgs = [f for f in os.listdir(img_dir) if f.endswith('.png')]
all_imgs.sort()
random.seed(42)
random.shuffle(all_imgs)

# 폴더 생성 함수
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def is_valid_label(lbl_path):
    try:
        with open(lbl_path) as fp:
            for line in fp:
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                if len(parts) < 5:
                    return False
                try:
                    cls_id = int(float(parts[0]))
                    nums = [float(x) for x in parts[1:]]
                except ValueError:
                    return False
                if cls_id < 0:
                    return False
                # bbox: x, y, w, h in [0,1]
                bbox = nums[:4]
                if any((x < 0.0 or x > 1.0) for x in bbox):
                    return False
                # keypoints (if present): x, y in [0,1], v in [0,2]
                if len(nums) > 4:
                    kpts = nums[4:]
                    if len(kpts) % 3 != 0:
                        return False
                    for i in range(0, len(kpts), 3):
                        x, y, v = kpts[i:i+3]
                        if x < 0.0 or x > 1.0:
                            return False
                        if y < 0.0 or y > 1.0:
                            return False
                        if v < 0.0 or v > 2.0:
                            return False
        return True
    except FileNotFoundError:
        return False

valid_imgs = []
invalid_imgs = []
for img in all_imgs:
    lbl_path = os.path.join(lbl_dir, img.replace('.png', '.txt'))
    if is_valid_label(lbl_path):
        valid_imgs.append(img)
    else:
        invalid_imgs.append(img)

n_total = len(valid_imgs)
n_train = int(n_total * 0.8)
train_imgs = valid_imgs[:n_train]
val_imgs = valid_imgs[n_train:]

# train/val 폴더에 이미지와 정상 레이블만 복사
for split, img_list in zip(['train', 'val'], [train_imgs, val_imgs]):
    img_out = os.path.join(base_dir, 'images', split)
    lbl_out = os.path.join(base_dir, 'labels', split)
    ensure_dir(img_out)
    ensure_dir(lbl_out)
    for img in img_list:
        lbl = img.replace('.png', '.txt')
        shutil.copy2(os.path.join(img_dir, img), img_out)
        shutil.copy2(os.path.join(lbl_dir, lbl), lbl_out)
print(f"train: {len(train_imgs)}개, val: {len(val_imgs)}개 (정상 레이블만 복사)")
if invalid_imgs:
    print(f"스킵: {len(invalid_imgs)}개 (레이블 오류/누락)")


train: 88개, val: 23개 (정상 레이블만 복사)


---

위 셀을 실행하면, 정상 레이블만 포함된 train/val 데이터셋이 자동으로 생성됩니다.
이제 아래 단계로 data.yaml 파일을 생성하세요.


## 3. 데이터 전처리
- data.yaml 파일을 생성하여 학습에 필요한 경로, 클래스 수, 클래스 이름을 정의합니다.

In [4]:
# data.yaml 파일 자동 생성 예시 (pose)

import yaml

data_yaml = {
    'train': f'{base_dir}/images/train',
    'val': f'{base_dir}/images/val',
    'kpt_shape': [17, 3],  # keypoints: 17개, (x, y, v)
    'nc': 2,  # 클래스 수 (예시: 낙상, 누움)
    'names': ['fall', 'lying']
}

with open(f'{base_dir}/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f, allow_unicode=True)
print('data.yaml 파일이 생성되었습니다.')


data.yaml 파일이 생성되었습니다.


## 4. 모델 설계 및 학습
- Ultralytics YOLOv8 모델을 불러와서, 생성한 data.yaml을 이용해 학습을 진행합니다.

In [5]:
# 캐시 제거 (레이블 변경 후 캐시가 남아있는 경우 오류 방지)
import os

cache_path = f'{base_dir}/labels/train.cache'
if os.path.exists(cache_path):
    os.remove(cache_path)
    print('cache 삭제 완료:', cache_path)
else:
    print('cache 없음')


cache 삭제 완료: /home/clyde/dev_ws/deeplearning-repo-2/data/CCTV_Incident_Fall_Lying_Dataset/laying_dataset/labels/train.cache


In [6]:
# Ultralytics YOLOv8 모델 불러오기 및 학습 (pose)
from ultralytics import YOLO

yaml_path = f'{base_dir}/data.yaml'
model = YOLO('yolov8n-pose.pt')  # pose 모델 사용

# epoch를 1로 설정하여 학습 (원하는 epoch로 직접 변경 가능)
epochs = 1  # 필요시 사용자가 직접 변경
print(f'\n==== {epochs} epoch로 학습 시작 ====')
results = model.train(
    data=yaml_path,
    epochs=epochs,
    imgsz=640,
    batch=4,
    project='runs/train',
    name=f'cctv_falllying_yolov8_pose_e{epochs}',
    exist_ok=True,
    device='cpu'  # GPU 사용 가능하면 'cuda'
    )
print(f'==== {epochs} epoch 학습 완료! ====')



==== 1 epoch로 학습 시작 ====
Ultralytics 8.3.240 🚀 Python-3.12.3 torch-2.9.1+cu128 CPU (11th Gen Intel Core i5-1135G7 @ 2.40GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/home/clyde/dev_ws/deeplearning-repo-2/data/CCTV_Incident_Fall_Lying_Dataset/laying_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=cctv_falllying_yolov8

## 5. 모델 평가
- 학습이 끝난 후, 주요 성능 지표를 요약 출력하고, 결과를 시각화합니다.

In [7]:
# 학습 결과 요약 및 시각화 (pose)
import pandas as pd
import os

# training 결과 저장 경로 (results 객체가 있으면 사용)
run_dir = None
if 'results' in globals() and hasattr(results, 'save_dir'):
    run_dir = results.save_dir

# fallback: 가장 최근 run 폴더 찾기
if not run_dir:
    run_base = os.path.join('runs', 'train')
    if os.path.isdir(run_base):
        runs = [os.path.join(run_base, d) for d in os.listdir(run_base)]
        runs = [d for d in runs if os.path.isdir(d)]
        if runs:
            run_dir = max(runs, key=os.path.getmtime)

if not run_dir:
    raise FileNotFoundError('학습 결과 폴더를 찾지 못했습니다.')

result_csv = os.path.join(run_dir, 'results.csv')
if os.path.exists(result_csv):
    df = pd.read_csv(result_csv)
    last = df.iloc[-1]
    # pose + box 지표 함께 출력 (존재하는 컬럼만)
    key_cols = [
        'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)',
        'metrics/precision(P)', 'metrics/recall(P)', 'metrics/mAP50(P)', 'metrics/mAP50-95(P)'
    ]
    print(f"[최종 Epoch: {int(last['epoch'])}]")
    for k in key_cols:
        if k in last:
            print(f"{k}: {last[k]:.4f}")
    # 곡선 시각화
    plot_cols = [c for c in key_cols if c in df.columns]
    if plot_cols:
        df[plot_cols].plot()
        plt.title('YOLOv8 Pose 학습 성능 지표 변화')
        plt.xlabel('Epoch')
        plt.ylabel('Score')
        plt.show()
else:
    print('결과 CSV 파일이 존재하지 않습니다:', result_csv)


[최종 Epoch: 1]
metrics/precision(B): 0.4561
metrics/recall(B): 1.0000
metrics/mAP50(B): 0.7424
metrics/mAP50-95(B): 0.5293
metrics/precision(P): 0.3950
metrics/recall(P): 0.8913
metrics/mAP50(P): 0.5750
metrics/mAP50-95(P): 0.2599


<Figure size 640x480 with 1 Axes>

## 6. 테스트 이미지 예측 및 결과 시각화
- 학습된 YOLO 모델로 테스트 이미지를 예측하고, 예측 결과를 시각화합니다.

In [8]:
# 테스트 이미지 예측 및 시각화
from PIL import Image

# 검증 이미지 폴더
test_img_dir = f'{base_dir}/images/val'
img_files = [f for f in os.listdir(test_img_dir) if f.endswith('.jpg') or f.endswith('.png')]

if not img_files:
    raise FileNotFoundError(f'No images found in: {test_img_dir}')

sample_img = os.path.join(test_img_dir, img_files[0])
results = model.predict(sample_img, save=False, imgsz=640, conf=0.25)

# 예측 결과 시각화
if hasattr(results[0], 'plot'):
    results[0].plot()
else:
    print('결과 시각화 기능은 ultralytics 버전에 따라 다를 수 있습니다.')



image 1/1 /home/clyde/dev_ws/deeplearning-repo-2/data/CCTV_Incident_Fall_Lying_Dataset/laying_dataset/images/val/laying65.png: 640x640 4 falls, 71.9ms
Speed: 2.3ms preprocess, 71.9ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


## 7. 모델 저장 및 불러오기
- 학습된 YOLO 모델을 파일로 저장하고, 저장된 모델을 다시 불러오는 방법을 안내합니다.

In [9]:
# 모델 저장 (YOLOv8은 자동으로 best.pt, last.pt 저장)
# 추가로 원하는 경로에 저장하려면 아래와 같이 사용
model_path = 'runs/train/cctv_falllying_yolov8_pose_e1/weights/best.pt'

# 모델 불러오기
loaded_model = YOLO(model_path)
print('저장된 모델을 성공적으로 불러왔습니다.')


저장된 모델을 성공적으로 불러왔습니다.
